# Getting Started Azure OpenAI with LangChain in Python

The purpose of this notebook is to provide a step-by-step guide on how to set up with Azure OpenAI and use the available models through the LangChain framework in Python. Additionally, it will demonstrate, through examples, how to implement models and utilize available features.

## Obtaining Keys and Endpoints

Using the [**Azure sign up page**](https://azure.microsoft.com/pricing/purchase-options/azure-account) one can sign up and create an Azure OpenAI resource, giving access to all necessary credentials.

## Setup
#### Installing and Importing Dependencies
Microsoft [**recommends**](https://github.com/microsoft/vscode-jupyter/wiki/Installing-Python-packages-in-Jupyter-Notebooks) using:

-  %pip for installing within Jupyter or IDE environments.
-  %conda for installing within Conda environments.
 

#### Install the most recent version of `langchain` and `langchain_openai`.

`%pip install -U langchain langchain_openai`

#### Install Packages in a Virtual Environment (Optional)
Set up a virtual environment by going to your project directory and executing the following command. This will create a new virtual environment in a folder named `.venv`.

**MacOS/UNIX**
- `python3 -m venv .venv`

**Windows**
- `py -m venv .venv`

#### Activating the Virtual Environment

To use the virtual environment, you must first activate it by executing the following command.

**MacOS/UNIX**
- `source .venv/bin/activate`

  
**Windows**
- `.venv\Scripts\activate`

#### Deactivating the Virtual Environment

If you want to leave the virtual environment in MacOS/UNIX and windows, simply execute the following command in the terminal:

`deactivate`

#### Set all required Environment Variables

Create a `.env` file and store your credentials like follows:

`AZURE_OPENAI_API_KEY` = <"Your key here">

`AZURE_OPENAI_ENDPOINT` = <"Your endpoint here">


#### Or use dotenv to set and load all required env variables into/from your `.env` file.

`%pip install python-dotenv`

In [1]:
from dotenv import set_key
set_key(".env", "OPENAI_API_VERSION", "Your api version here")
set_key(".env", "COMPLETIONS_MODEL", "Your model here")

(True, 'COMPLETIONS_MODEL', 'Your model here')

#### Get all required Environment Variables

In [2]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [3]:
import os

# Setting up the deployment name
DEPLOYMENT_NAME = os.environ['COMPLETIONS_MODEL']

# The API key for your Azure OpenAI resource.
API_KEY = os.environ['AZURE_OPENAI_API_KEY']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
ENDPOINT = os.environ['AZURE_OPENAI_ENDPOINT']

# The API version required
VERSION = os.environ['OPENAI_API_VERSION']

## Creating an AzureChatOpenAI Model

More information on LangChain's AzureChatOpenAI support can be found in [the integration documentation](https://python.langchain.com/v0.2/docs/integrations/chat/azure_chat_openai/).

In [5]:
from langchain_openai import AzureChatOpenAI

- Environment variable values can be passed as parameters.
- Alternatively, if not passed in, the constructor will search for environment variables with corresponding names.

In [12]:
model = AzureChatOpenAI(
    openai_api_version=VERSION,  
    azure_deployment=DEPLOYMENT_NAME,
    azure_endpoint=ENDPOINT,
    temperature=0.5,
    max_tokens=200,
    timeout=60,
    max_retries=10,
    # model="gpt-35-turbo",
    # model_version="0125",
    # other params...
    )

In the above code sample, **OPENAI_API_VERSION** and **AZURE_OPENAI_ENDPOINT** are both being passed in, but **AZURE_OPENAI_API_KEY** is being retrieved within the constructor.

#### Other Optional Parameters

- `temperature` determines how creative and unpredictable, or how deterministic and predictable, the model should be in its responses. A temperature of 0 would be predictable, while anything higher would make responses more random.

- `max_tokens` defines the maximum number of tokens (words or pieces of words) the model can generate in its response.

- `timeout` specifies the maximum amount of time (in seconds) to wait for a response from the API before timing out.

- `max_retries` sets the number of times the API request should be retried in case of failure before giving up.

- `model` specifies the model to be used.

- `model_version` indicates the specific version of the chosen model to use.

- See the [**API Reference**](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.azure.AzureChatOpenAI.html) for more details.

- Other parameters may be available in different SDK's.

## Model Usage

In [51]:
from langchain_core.messages import HumanMessage, SystemMessage

#### Using Messages from the `langchain_core.messages` Library

The `langchain_core.messages` library allows the user to define messages for the model and assign roles to each message.

- LangChain-compatible chat models take a list of `messages` as `input` and return the AI message as `output`.

- All messages have `role` and `content` properties. In the sample below, the roles are set by using the `SystemMessage` and `HumanMessage` classes. [**We'll cover more on this later**](#assigning-roles) .

- Additional provider-specific information can be incorporated using the `Additional Properties`.

In [13]:
messages = [
    SystemMessage(content="Translate the following from German into English"),
    HumanMessage(content="Sie haben gerade Ihr erstes Kunstliche Itelligenz Model erstellt!"),
]

In [14]:
response = model.invoke(messages)

In [32]:
response.content

'You have just created your first artificial intelligence model!'

### Prompting

- Prompts are the inputs to language models, refined from raw user inputs to be ready for processing by the models.

- [**Prompting**](https://www.datacamp.com/tutorial/prompt-engineering-with-langchain) involves crafting text inputs that clearly communicate with the models, outlining the specific task we want it to accomplish. This can include:
    - Selecting the appropriate wording and setting a particular tone or style.
    - Providing necessary context.
    - Assigning a role, such as asking it to respond as if it were a native speaker of a certain language.

#### Prompt Templates

- LangChain allows developers to design parameterized [**Prompt Templates**](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/) that are reusable and easily transferable between different models for integration.

- It takes user input and inserts said input into the prompt to feed into the language models.

#### `PromptTemplate`

`PromptTemplate` is used to create an instance of [**Prompt**](https://python.langchain.com/v0.1/docs/expression_language/get_started/#1-prompt), and this is `invoked` by sending it to a model, which produces a `PromptValue`.

The example code uses `.from_template`, which handles a single string template with placeholders for dynamic inputs.

In [15]:
from langchain_core.prompts import PromptTemplate  

prompt_template = PromptTemplate.from_template(
    "What vegetable crops can I grow in {month} in {city}, New Zealand?"
)

prompt_value = prompt_template.format(month = "December", city = "Rotorua")


# print(prompt_template) # <- uncomment to see
# print(prompt_value) # <- uncomment to see

In [16]:
response = model.invoke(prompt_value)
response.content

"In Rotorua, New Zealand, December falls in the Southern Hemisphere's summer, which is a great time for growing a variety of vegetables. Here are some vegetable crops you can plant in December:\n\n1. **Tomatoes**: Ideal for summer planting, they thrive in the warm weather.\n2. **Capsicums (Bell Peppers)**: These also enjoy the summer heat.\n3. **Zucchini**: Fast-growing and productive during warm months.\n4. **Cucumbers**: Perfect for summer salads and pickling.\n5. **Beans**: Both bush and pole beans grow well in the warm season.\n6. **Sweet Corn**: Requires warm temperatures and plenty of sunlight.\n7. **Pumpkins**: Plant now for a harvest in autumn.\n8. **Eggplants**: Another heat-loving crop.\n9. **Lettuce**: Opt for heat-tolerant varieties to avoid bolting.\n10. **Radishes**: Fast-growing and can"

#### `ChatPromptTemplate`

This is optimized for a conversation-like format. The prompt is a list of chat messages. Each chat message is associated with `role` and `content`. In the example code, `.from_messages` is used to include multiple messages.

Here, we will hardcode roles in the chat prompt, as opposed to using the pre-built roles `SystemMessage` or `HumanMessage` like earlier.

In [17]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
                You're a travel agent helping customers plan their trips.
                Offer recommendations on natural features to visit, local cuisine, and activities based on the country the customer is asking about.
               """),
        ("ai", "Hi there, What can I help you with today?"),
        ("human", "Hi I'm {name}, I'm planning a trip to {country}. Any recommendations")
    ]
)

prompt_value = chat_template.format_messages(name="Lucy", country="New Zealand")

# print(chat_template) # <- uncomment to see
# print(prompt_value) # <- uncomment to see

In [18]:
response = model.invoke(prompt_value)
response.content

"Hi Lucy! New Zealand is a fantastic choice with its stunning landscapes, rich culture, and exciting activities. Here are some recommendations to make your trip memorable:\n\n### Natural Features\n1. **Fiordland National Park**: Home to the famous Milford Sound and Doubtful Sound, this area offers breathtaking fjords, waterfalls, and rainforests.\n2. **Tongariro National Park**: Known for its dramatic volcanic landscape, you can hike the Tongariro Alpine Crossing, one of the best one-day hikes in the world.\n3. **Rotorua**: Famous for its geothermal activity, you can see geysers, hot springs, and mud pools. Don't miss the Wai-O-Tapu Thermal Wonderland.\n4. **Aoraki/Mount Cook**: The highest mountain in New Zealand, offering stunning views, glaciers, and excellent hiking trails.\n5. **Bay of Islands**: A beautiful coastal area with over 140 subtropical islands, perfect for sailing, fishing,"

<a id="assigning-roles"></a>
#### Assigning Roles Using LangChain Messages

Compared to hardcoding the roles like above, LangChain Messages allow for more flexibility and better management, especially with complex conversations involving multiple roles. It also simplifies the visualization of the conversation flow.

It is therefore recommended to use LangChain messages where possible.

**Basic Message Types**

|             |                                                                 |
|-------------|-----------------------------------------------------------------|
| `SystemMessage` | Set how the AI should behave (appropriate wording, tone, style, etc.) |
| `HumanMessage`  | Message sent from the user                                      |
| `AIMessage`     | Message from the AI chat model (context setting, guidance for response) |

For more info, see [**Message Types**](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/) and [**API Reference**](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.messages).

#### `base message` and `MessagePromptTemplate`
We can also pass a `base message` or `MessagePromptTemplate` instead of tuples.

In [47]:
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("You are a translator. You are to translate the text into English."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

prompt_value = chat_template.format_messages(text="アサヒスーパードライは日本のビールのです。")

# print(chat_template) # <- uncomment to see
# print(prompt_value) # <- uncomment to see

In [48]:
response = model.invoke(prompt_value)
response.content

'Asahi Super Dry is a Japanese beer.'

#### `MessagePlaceHolder`
This is used to select which messages to include when formatting.

In [50]:
from langchain_core.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
    AIMessagePromptTemplate,
)

# SYSTEM ROLE Prompt
system_template = SystemMessagePromptTemplate.from_template("""
                                            You are a precise assistant who knows the shcedule of the team.
                                            Schedule details are as follows: {schedule}.
                                            Only provide information to the team members.
                                            Strictly only provide information specific to what is asked, Do not give extra information.
                                            """)
# HUMAN ROLE Prompt
human_template = HumanMessagePromptTemplate.from_template("My name is {user_name}.")
# AI ROLE Prompt
ai_template = AIMessagePromptTemplate.from_template("Hello {user_name}, how can I help you today?")

chat_prompt = ChatPromptTemplate.from_messages(
    [
        # this has essentially created a 'conversation history'
        system_template,
        human_template,
        ai_template,
        MessagesPlaceholder(variable_name="conversation"), 
    ]
)

# print(chat_prompt) # <- uncomment to see the chat prompt 

We can then input more prompts, which will take the `MessagePlaceholders`' place and create lines of sentences or a conversation.

In [52]:
from langchain_core.messages import HumanMessage, AIMessage

schedule = """
    Team Members: Alice, Bob, Carol, David, Emily
    Team Meeting Schedule: Every Tuesday at 11:00 AM
    Topic: LangChain with Azure OpenAI Integration
"""
# these messages will take MESSAGEPLACEHOLDERS place
human_query = HumanMessage("When is the next team meeting and who is attending?")
ai_message = AIMessage("Hold on a second, let me check the schedule for you.")

prompt_value = chat_prompt.format_messages(
    conversation=[human_query, ai_message], user_name="David", schedule=schedule
)

# print(prompt_value) # <- uncomment to see the prompt 

In [53]:
response = model.invoke(prompt_value)
response.content

'The next team meeting is on Tuesday at 11:00 AM. The attendees are Alice, Bob, Carol, David, and Emily.'

#### `FewShotPrompt`

We can use examples (shots) to condition the model for a better response by including some example input and output in the prompt. This will inform the model about the context and how we want the output to be formatted.

In [55]:
from langchain_core.prompts import FewShotPromptTemplate

examples = [
    {"input": "one dollar", "output": "$1"},
    {"input": "thirty five euros", "output": "€35"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"], template="Currency Unit Conversion: [Input] {input} => [Output] {output}"
)

# unpack the first example dictionary and feed it to the prompt template to format
print(example_prompt.format(**examples[0]))

# feed examples to FewShotPromptTemplate to generate a final prompt
fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix= "Convert the currency units: {input}",
    input_variables=["input"],
)

prompt_value= fewshot_prompt.format(input="one hundred yen")

response = model.invoke(prompt_value)
print(response.content)

Currency Unit Conversion: [Input] one dollar => [Output] $1
Currency Unit Conversion: [Input] one hundred yen => [Output] ¥100


## Chaining

- Many LangChain components implement the [**Runnable**](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface) protocol, which allows them to be easily chained together. These components can be combined in a sequence of calls, which we refer to as a chain.

- Chaining `Runnables` in sequence.

In [57]:
system_template = SystemMessagePromptTemplate.from_template("""
    You are an expert in {country} cuisine. 
    Keep it simple and short.
    """)

chat_prompt = ChatPromptTemplate.from_messages(
    [
         system_template,
        ("human", "I'd like to find out about {country} cuisine."),
        ("human", "{question}"),
    ]
)

This is how we have been using prompts, but now we will skip this step and invoke using the chain.

In [59]:
prompt_value = chat_prompt.format_messages(country="Japanese", question="What is the most popular Sashimi in Japan vs the rest of the world?")

response = model.invoke(prompt_value)
response.content

'In Japan, maguro (tuna) is the most popular sashimi. Globally, salmon sashimi tends to be more popular.'

In [60]:
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | model | StrOutputParser()

print(
    chain.invoke(
        {
            "country": "Japanese", 
            "question": "What is the most popular Sashimi in Japan vs the rest of the world?"
        }
    )
)

In Japan, the most popular sashimi is often Maguro (tuna), specifically the fatty part known as Otoro. Globally, Salmon sashimi tends to be more popular due to its rich flavor and widespread availability.


### Wait in between API calls

Depending on the users Azure plan and account will determine how many API requests can be made within a given interval. If too many requests are sent frequently an error may occur where the user will be told to wait **x** amount of time before sending another request.

When creating a model, one of the parameters briefly mentioned earlier was the `max_retries` parameter. When calling the underlying Python OpenAI library, the library will wait and retry the call on your behalf at least 2 times (the default) before raising a RateLimiteError, or whatever `max retries` is set as.

Otherwise, to ensure the user is not overwhelming the Azure services, make sure to wait between API calls when possible - this can be enforced by using the `time` package.

In [65]:
import time
time.sleep(5)

### Check the costs and token usage of a given model API call

In [49]:
from langchain_community.callbacks import get_openai_callback

In [52]:
messages = [
    SystemMessage(content="Translate the following from German into English"),
    HumanMessage(content="What's the first planet in the Solar System?"),
]

`get_openai_callback()` is a context manager of the [**OpenAICallbackHandler**](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/callbacks/openai_info.py) class, meaning it calls this class and creates an instance when used.

Below is an example of how to use the `get_openai_callback()`. However, to get an accurate estimation of cost, you must pass the model and model version as parameters to the `AzureChatOpenAI` [**constructor**](#creating-an-azurechatopenai-model). 

In [53]:
with get_openai_callback() as cb:
    model.invoke(messages)
    print(f"Total tokens used: {cb.total_tokens}")
    print(f"Total prompt tokens: {cb.prompt_tokens}")
    print(f"Total prompt tokens: {cb.completion_tokens}")
    print(f"Total cost (in dollars): ${cb.total_cost}")
    print(f"Total successful requests): {cb.successful_requests}")

Total tokens used: 37
Total prompt tokens: 27
Total prompt tokens: 10
Total cost (in dollars): $0.000285
Total successful requests): 1


## How to use structured outputs

Import the required packages. `BaseModel` is a parent class that all tools will inherit from, and `Field` is used to define all properties of the tool.

In [20]:
from langchain_core.pydantic_v1 import BaseModel, Field

#### Tools

Tools are essentially classes that can be passed to a chosen model to influence or structure how the response should be formatted or generated.

**For example:**

- A Weather tool with a specific API call could be passed so the model knows to use this specific API for data retrieval.
- A City tool with fields like `population`, `size`, and `main_language` so the model can return any city-related queries with an object containing the corresponding filled fields.
- An Image tool with a `url` field to be returned when asked to search for an image containing a dog, with the field containing the URL of the image.

In [43]:
from typing import Optional
#adds the ability to make a field Optional [NULL]

class Person(BaseModel):
    '''Information about a given person'''

    name: str = Field(..., description="The name of a person")
    alive: bool = Field(..., description="Whether the person is alive or not")
    place_of_birth: str = Field(..., description="Where the person was born")
    noteable_features: str = Field(..., description="Any noteworthy features/achievements about the person")
    hobbies: str = Field(..., description="Any hobbies the person may have")

In [44]:
structured_model = model.with_structured_output(Person)
response = structured_model.invoke("Tell me about Kate Sheppard")
response

Person(name='Kate Sheppard', alive=False, place_of_birth='Liverpool, England', noteable_features="Leader of the women's suffrage movement in New Zealand, instrumental in making New Zealand the first country to grant women the right to vote in 1893.", hobbies='Activism, writing, public speaking')

##### As the response of the invocation has been structured using the Person tool, the response can be accessed like a `Person` object.

In [45]:
print(response.name)
print(response.alive)
print(response.place_of_birth)
print(response.noteable_features)

Kate Sheppard
False
Liverpool, England
Leader of the women's suffrage movement in New Zealand, instrumental in making New Zealand the first country to grant women the right to vote in 1893.


#### JSON

In [8]:
import json

Models can also be explicitly told to respond in a JSON structured format. This could then be used for future API calls or for easier access to information. However, **the word "json" must be included in the message string.**

In [46]:
json_model = model.bind(response_format={"type": "json_object"})

response = json_model.invoke(
    '''Return a JSON object of a random person with features like name,
    alive (if they're alive or not) and their place of birth.''' 
)

response.content

'{\n    "name": "Jane Doe",\n    "alive": true,\n    "place_of_birth": "Springfield, Illinois, USA"\n}'

The response can then be formatted into a JSON object and accessed using normal JSON notation.

In [47]:
person = json.loads(response.content)
person

{'name': 'Jane Doe',
 'alive': True,
 'place_of_birth': 'Springfield, Illinois, USA'}

## File input

Models can be fed files of various forms as their inputs.

#### Images

Import the required libraries to allow for HTTP requests and file conversion.

- Make sure the file format is correct; otherwise, a "File not found" error will be thrown.

- Ensure the HTTP request was successful with a 200 response code; otherwise, a 404 error will be thrown.

In [58]:
import base64
import httpx

url = "https://upload.wikimedia.org/wikipedia/commons/b/bf/Aoraki_Mount_Cook.JPG"

# Use an HTTP GET request to retrieve the image
response = httpx.get(url)

# Check the request was successful
if response.status_code == 200:
    # Extract the binary image data
    binary_image_data = response.content

    # Encode the binary_image_data into base 64 so that it's in bytes
    # This allows it to be transmitted in JSON or text format
    byte_image_data = base64.b64encode(binary_image_data)

    # Decode the data into UTF-8 to allow for to make the data workable
    # Web applications and models require it in a workable format
    # UTF-8 is a fairly standardised format
    image_data = byte_image_data.decode("utf-8")

print(f'The status code is: {response.status_code}')

The status code is: 200


When using data of different types, such as text in the `SystemMessage` and a file in the `HumanMessage`, it's necessary to specify a type header so the model knows how to interpret the data.


In [60]:
messages = [  
    SystemMessage(content=[{"type": "text", "text": "describe the image location"}]),
    HumanMessage(
    content=[
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
    )
]
response = model.invoke(messages)
response.content

'This image depicts a stunning mountainous landscape with a snow-capped peak reflected in a calm, clear lake. The mountains are rugged and covered in snow, suggesting a cold and possibly remote location. The clear blue sky and the reflection in the water add to the serene and picturesque quality of the scene. The foreground features some rocks along the edge of the lake, enhancing the natural beauty of the setting. This type of scenery is often found in high-altitude regions, such as the Himalayas, the Alps, or the Southern Alps in New Zealand.'

#### Audio

Chat completion models are somewhat limited in how they work with audio files. Audio files cannot be transmitted to the model directly but must instead be transcribed using the [**Whisper API**](https://platform.openai.com/docs/guides/speech-to-text). The transcribed text can then be used with the model.


The Whisper API can be accessed through `langchain_community` services.


In [36]:
from langchain_community.document_loaders.parsers import OpenAIWhisperParser

## Embeddings

Embeddings, particularly `AzureOpenAIEmbeddings`, are a natural language processing technique that converts text into mathematical or vector representations. These representations capture the semantic meaning of words, phrases, or entire texts. This transformation enables Azure OpenAI search services to utilize numerical similarities between texts, returning the most relevant search results for a given query.

#### Setup

Embeddings models use a different Azure resource, this model name will be set below as follows.

In [18]:
EMBEDDINGS_MODEL = os.environ['EMBEDDINGS_MODEL']

In [20]:
from langchain_openai import AzureOpenAIEmbeddings

#### Create a model

In [73]:
E_model = AzureOpenAIEmbeddings(model = EMBEDDINGS_MODEL, azure_endpoint=ENDPOINT)

# random generated text
text = ["The sun sets behind the mountains, casting a warm glow over the valley below.",
        "Advances in artificial intelligence are transforming industries across the globe.",
        "The recipe calls for two cups of flour, one teaspoon of baking powder, and a pinch of salt.",
        "Exercise is essential for maintaining a healthy body and mind, promoting overall well-being.",
        "Space exploration continues to reveal the mysteries of the universe, pushing the boundaries of human knowledge.",
        "The novel’s protagonist faces a moral dilemma that challenges their deepest beliefs.",
        "Sustainable practices in agriculture are crucial for preserving our environment for future generations."
       ] 

embeddings = E_model.embed_documents(text) 
print(embeddings[0][:5])

[0.029870394617319107, -0.004167019855231047, 0.008153270930051804, -0.011176464147865772, -0.015433868393301964]


#### Searching

To leverage the power of embeddings we will transform the text into an easily accessable data structure known as a `DataFrame` from the `pandas` library.

`%pip install pandas`

In [75]:
import pandas as pd

In [77]:
df_text = pd.DataFrame(text)
df_text

,0
0,"The sun sets behind the mountains, casting a w..."
1,Advances in artificial intelligence are transf...
2,"The recipe calls for two cups of flour, one te..."
3,Exercise is essential for maintaining a health...
4,Space exploration continues to reveal the myst...
5,The novel’s protagonist faces a moral dilemma ...
6,Sustainable practices in agriculture are cruci...


Rename the columns and index to make the `DataFrame` clearer.

In [79]:
df_text.rename(columns={0:'text'}, inplace=True)
df_text.index.name = "index"

In [81]:
df_text

,text
index,
0,"The sun sets behind the mountains, casting a w..."
1,Advances in artificial intelligence are transf...
2,"The recipe calls for two cups of flour, one te..."
3,Exercise is essential for maintaining a health...
4,Space exploration continues to reveal the myst...
5,The novel’s protagonist faces a moral dilemma ...
6,Sustainable practices in agriculture are cruci...


In [83]:
df_text['Embeddings'] = df_text['text'].apply(E_model.embed_query)

In [85]:
df_text

,text,Embeddings
index,,
0,"The sun sets behind the mountains, casting a w...","[0.029870394617319107, -0.004167019855231047, ..."
1,Advances in artificial intelligence are transf...,"[-0.012673170305788517, -0.020036686211824417,..."
2,"The recipe calls for two cups of flour, one te...","[0.012168226763606071, 0.018271654844284058, 0..."
3,Exercise is essential for maintaining a health...,"[-0.01102688442915678, -0.007886004634201527, ..."
4,Space exploration continues to reveal the myst...,"[0.028899280354380608, -0.01474663894623518, -..."
5,The novel’s protagonist faces a moral dilemma ...,"[0.013899882324039936, -0.02602808177471161, -..."
6,Sustainable practices in agriculture are cruci...,"[-0.006976415403187275, -0.018390081822872162,..."


Here you can see that for each `text` it's corresponding embeddings have been set in the `Embeddings` column in the same row.

In [89]:
E_model.embed_query(text[1])[:2]

[-0.012673170305788517, -0.020036686211824417]

Now that we have the numerical representation, various Azure services can use this for searching. However, to demonstrate what happens 'under the hood,' we will implement a basic vector search manually.

#### Cosine similarity

Cosine similarity measures the similarity between two vectors by evaluating the cosine of the angle between them. In essence, it determines how close two vector points or lines are to each other. Vectors that are closer in space typically share a closer semantic meaning according to the model. This principle forms the core functionality of vector-based search using embeddings.

#### Setup

The `numpy` library introduces some conveniant mathematical functions.

`%pip install numpy`

In [98]:
import numpy as np
from numpy.linalg import norm

In [100]:
def cosine_similarity(text, query):
    return np.dot(text, query) / (np.linalg.norm(text) * np.linalg.norm(query))

#### Search

Use embeddings to get the vector representation of a query.

In [105]:
query = 'What text talks about space or stars?'
query_embedding = E_model.embed_query(query)

In [111]:
df_text['Similarity'] = df_text['Embeddings'].apply(lambda text_embedding : cosine_similarity(text_embedding, query_embedding)) 

In [119]:
df_text.sort_values(by='Similarity', ascending=False).head(2)

,text,Embeddings,Similarity
index,,,
4,Space exploration continues to reveal the myst...,"[0.028899280354380608, -0.01474663894623518, -...",0.825995
0,"The sun sets behind the mountains, casting a w...","[0.029870394617319107, -0.004167019855231047, ...",0.759168


Et voilà! The text with the highest cosine similarity discusses space and space exploration, which closely aligns with our query. We also observe that the second-most similar text mentions the Sun. While it’s not an exact match to our query, the Sun is a star, making it contextually similar and resulting in a relatively high cosine similarity.